<a href="https://colab.research.google.com/github/pphos/pytorch_tutorial/blob/master/neural_network_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NEURAL NETWORKS
ニューラルネットワークは`torch.nn`パッケージを用いて構築することができます.

`nn.Module`にはレイヤーと, 出力を返す`forward(input)`メソッドがあります.

ニューラルネットワークの典型的な学習手順は次の通りです.

- 学習可能なパラメータを持つニューラルネットワークを定義する
- 入力データセットを繰り返し処理する
- ネットワークを介して入力を処理する
- 損失を計算する (損失誤差がどれだけ小さく正解に近いかを計算する)
- 勾配をネットワークに逆伝搬させる
- ネットワークの重みを更新する:

 `weight = wehght - learning_rate * gradient`

## Define the network

ネットワークを定義してみよう:

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    # 1 input image channel, 6 output channels, 5x5 square convolution
    # kernel
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    # an affine operation: y = Wx + b
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    # Max pooling over a (2, 2) window
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    # If the size is a square you can only specify a single number
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:]
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


`forward`関数を定義するだけで, 勾配の計算を行う`backward`関数は`autograd`により自動的に定義されます. `forward`関数では, Tensor演算のいずれかを用いることができます.

学習可能なモデルのパラメータは`net.parameters()`によって返されます.

In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


32x32のランダムな入力を試してみましょう.

Note:
このネットワーク(LeNet)に期待される入力サイズは32x32です. MNISTデータセットをLeNetに適用するためには, データセットの画像を32x32にリサイズする必要があります.

In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0512, -0.0822,  0.0266, -0.0203, -0.1145,  0.0082, -0.0506,  0.0111,
          0.0776,  0.0218]], grad_fn=<AddmmBackward>)


全てのパラメータとランダムな勾配を持つ逆伝搬の勾配のバッファを0にします.

In [0]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### Note
``torch.nn``はミニバッチ学習のみをサポートしています.
``torch.nn``パッケージ全体では, サンプルのミニバッチ入力のみをサポートしており, 1サンプルの入力はサポートしていません.

先に進む前に、これまでの内容をまとめてみましょう.

Recap:
- `torch.Tensor` - 多次元配列で``backward()``のような自動微分をサポートしています. また, テンソルに関連した勾配も保持します.
- `nn.Module` - ニューラルネットワークモジュールおよびパラメータをカプセル化する便利な手段であり, GPUへの移動, エクスポート, ロードなどのヘルパがあります.
- `nn.Parameter` - Tensorの一種で, `Module`に属性として代入すると自動的にパラメータとして登録されます.
- `autograd.Function` - 自動微分の順伝搬および逆伝搬の定義を実装します.
すべての`Tensor`操作は少なくとも一るの`Function`ノードを生成し, そのノートは`Tensor`を生成した関数に接続し, その履歴をエンコードします.


## Loss Function
損失関数は（出力, 目標)の入力のペアを受け取り, 
出力が目標からどれだけ離れているかを推定する値を計算します.
``nn``パッケージには, 異なる損失関数が複数実装されています.

単純な損失関数は``nn.MSELoss``です. これは入力と目標の間の平均二乗誤差を計算します.

In [6]:
out = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(out, target)
print(loss)

tensor(1.0076, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

# Backprop
誤差逆伝搬するためには`loss.backward()`を実行するだけです.
この時, 既存の勾配情報に新たな情報を蓄積しないために, 既存の勾配情報を初期化しておく必要があります.

ここでは, `loss.backward()`を呼び出して, 逆伝搬の前後でconv1s の勾配を見ることにします.

In [8]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0086, -0.0079,  0.0199, -0.0061, -0.0107, -0.0025])


## Update the weights
実際に使用されている最も単純な最適化関数は確率的勾配降下法(Stochastic Gradient Descent: SGD)です.

```
    weight = weight - learning_rate * gradient
```
SGDはPythonを使って以下のように実装できます.

In [0]:
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

しかし、ニューラルネットワークを使用すると, SGD, Nesterov-SGD, Adam, RMSPropなど, さまざまな最適化関数を使用したくなります.
これを可能にするために, これらの最適化関数を実装した`torch.optim`があります. この使い方はとても簡単で以下の通りです.

In [0]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()